In [2]:
import sys, os
sys.path.append(os.path.join(os.path.dirname('__file__'), '..', 'DB_and_Azure'))
import sql_db_functions as SQLf


### get data from server

In [47]:
conn, cursor = SQLf.sql_db_functions.connect_sql()

In [48]:
query = f"SELECT * FROM Products ;" #LIMIT 5
cursor.execute(query)

# Fetch the rows
rows = cursor.fetchall()

In [49]:
import pandas as pd

In [50]:
df = pd.DataFrame(data=rows, columns=['Brand_Prod_ID','Brand','Descript','Price','link','Clothing_type'])
df.head()

,Brand_Prod_ID,Brand,Descript,Price,link,Clothing_type
0,1,Gucci,Il monogramma GG è riproposto in versioni e de...,1500.0,https://www.gucci.com/it/it/pr/women/ready-to-...,Top e Camicie Donna
1,2,Gucci,Il monogramma GG è riproposto in versioni e de...,650.0,https://www.gucci.com/it/it/pr/women/ready-to-...,Top e Camicie Donna
2,3,Gucci,Ispirato allo spirito estivo e ai beach club d...,720.0,https://www.gucci.com/it/it/pr/women/ready-to-...,Top e Camicie Donna
3,4,Gucci,Ispirato allo spirito estivo e ai beach club d...,1100.0,https://www.gucci.com/it/it/pr/women/ready-to-...,Top e Camicie Donna
4,5,Gucci,Ispirato allo spirito estivo e ai beach club d...,1100.0,https://www.gucci.com/it/it/pr/women/ready-to-...,Top e Camicie Donna


In [51]:
df['link'] = df['link'].apply(lambda x: x if 'https' in x else 'https:' +  x )

In [52]:
df.head()

,Brand_Prod_ID,Brand,Descript,Price,link,Clothing_type
0,1,Gucci,Il monogramma GG è riproposto in versioni e de...,1500.0,https://www.gucci.com/it/it/pr/women/ready-to-...,Top e Camicie Donna
1,2,Gucci,Il monogramma GG è riproposto in versioni e de...,650.0,https://www.gucci.com/it/it/pr/women/ready-to-...,Top e Camicie Donna
2,3,Gucci,Ispirato allo spirito estivo e ai beach club d...,720.0,https://www.gucci.com/it/it/pr/women/ready-to-...,Top e Camicie Donna
3,4,Gucci,Ispirato allo spirito estivo e ai beach club d...,1100.0,https://www.gucci.com/it/it/pr/women/ready-to-...,Top e Camicie Donna
4,5,Gucci,Ispirato allo spirito estivo e ai beach club d...,1100.0,https://www.gucci.com/it/it/pr/women/ready-to-...,Top e Camicie Donna


In [53]:
SQLf.sql_db_functions.close_connection_db(conn=conn, cursor=cursor)

### look for product image

In [54]:
current_product_ID = df.iloc[0].Brand_Prod_ID
current_product_ID

1

In [55]:
conn, cursor = SQLf.sql_db_functions.connect_sql()

In [56]:
query = f"SELECT * FROM product_img WHERE Brand_id = {current_product_ID} ;"
cursor.execute(query)

# Fetch the rows
image_links = cursor.fetchall()

In [57]:
SQLf.sql_db_functions.close_connection_db(conn=conn, cursor=cursor)

In [58]:
df_images = pd.DataFrame(data=image_links, columns=['id','Brand_id','image_link'])
df_images.head()

,id,Brand_id,image_link
0,1,1,//media.gucci.com/style/DarkGray_Center_0_0_98...
1,2,1,//media.gucci.com/style/DarkGray_Center_0_0_98...
2,3,1,//media.gucci.com/style/DarkGray_Center_0_0_98...
3,4,1,//media.gucci.com/style/White_Center_0_0_730x4...


In [59]:
df_images['image_link'] = df_images['image_link'].apply(lambda x: x if 'https' in x else 'https:' +  x )

### Download image

In [60]:
df_images.head()

,id,Brand_id,image_link
0,1,1,https://media.gucci.com/style/DarkGray_Center_...
1,2,1,https://media.gucci.com/style/DarkGray_Center_...
2,3,1,https://media.gucci.com/style/DarkGray_Center_...
3,4,1,https://media.gucci.com/style/White_Center_0_0...


In [61]:
current_description = df[df.Brand_Prod_ID == current_product_ID]['Descript']
current_description

0    Il monogramma GG è riproposto in versioni e de...
Name: Descript, dtype: object

In [62]:
current_images = df_images.image_link
current_images

0    https://media.gucci.com/style/DarkGray_Center_...
1    https://media.gucci.com/style/DarkGray_Center_...
2    https://media.gucci.com/style/DarkGray_Center_...
3    https://media.gucci.com/style/White_Center_0_0...
Name: image_link, dtype: object

### Creating prompt

In [63]:
prompt_instructions = """

    The images are of the same clothing peace, use them to fill the following format to list attributes of the clothing peace in image, separate them with comma:  

    *Type:  
    *Fit: (Slim fit, regular fit, oversized, tight, loose, etc)
    *Sleeve style: (Short sleeve, long sleeve, sleeveless, cap sleeve, etc)     
    *Neckline: (Crew neck, V-neck, scoop neck, etc)   
    *Material: (Cotton, linen, silk, polyester, etc)
    *Formality: (Casual, Business Casual, Formal, etc)
    *Seasson:   
    *Colors:  (percentage of each color in the clothing peace) 
    *Texture: (Smooth, Rough, Ribbed, Velvety, etc)
    *Transparency: (Opaque, Semi-transparent, Transparent, etc)
    *Details and Embellishments: (e.g, buttons, zippers, embroidery)
    *Shape: (e.g., boxy, fitted, flared) 
    *Length: (e.g., cropped, hip-length, tunic)
    *Collar Style: (e.g., crew neck, V-neck, polo, button-down)
    *Sleeve Style: (e.g., short, long, three-quarter, sleeveless, puffed)
    *Patterns: (e.g., stripes, floral, geometric, not applicable)
    *Patterns placement: (e.g., lower back, left sholder, right chest, not applicable) 
    *Fluidity of fabric:
    *Fabric weight: (Light, medium, or heavy)
    *Pocket Presence:(Yes,No)
    *Pocket placement:(e.g., lower back, left sholder, right chest, not applicable)
    *Pocket size:(small, medium, big, not applicable)
    *Breathability: (Low, Medium, High)
    *Occasion Suitability: (Casual, formal, business casual, etc)
    *Lapel: (Notch, Peak, Shawl Satin, Slim, Wide, Contrasting, etc)


    Use information from the image and the description.
    IGNORE BACKGROUND
    ONLY ANSWER THE FORMAT  
    DONT USE THE BRAND NAME IN ANY DESCRIPTION  
    IN EACH DESCRIPTION - CHARACTERISTIC SHOULD BE ITS OWN DESCRIPTION
    YOU CAN USE MULTIPLE DESCRIPTIONS IN A SINGLE ATTRIBUTE  
    DONT USE MULTICOLOR OR Multicolor, NAME THE SPECIFIC COLORS  
    COLORS SHOULD INCLUDE PRINT AND CLOTHE COLORS WITHOUT ANY DESCRIPTION OF THE PRINT ONLY MAIN COLORS
   
"""

In [64]:
def generate_prompt(image_urls, description,prompt_instructions):
    # Construct the image part of the prompt
    images_text = ' '.join([f"{url}" for url in image_urls])
    # Combine with the description
    prompt_text = f"""
        {images_text} \n
        Description: {description} \n
        
        {prompt_instructions}
    """
    return prompt_text

In [65]:
prompt_text = generate_prompt(current_images, current_description,prompt_instructions)

### Send to chatGPT

In [66]:
import getpass
from apikey import apikey 

In [67]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
 

In [68]:
os.environ['OPENAI_API_KEY'] = apikey

In [69]:
# LLM
model = ChatOpenAI(model="gpt-4o", temperature=0.7)

In [70]:
# Prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a fashion specialist."),
        ("user", prompt_text),
    ]
)

In [71]:
chain = prompt | model

In [72]:
response = chain.invoke({})
print(response.content)

KeyboardInterrupt: 

In [ ]:
cloth_form = response.content

In [ ]:
cloth_form

'*Type: Shirt  \n*Fit: Regular fit  \n*Sleeve style: Long sleeve  \n*Neckline: Button-down  \n*Material: Silk  \n*Formality: Business Casual  \n*Season: All seasons  \n*Colors: 50% pink, 50% white  \n*Texture: Smooth  \n*Transparency: Opaque  \n*Details and Embellishments: Buttons  \n*Shape: Fitted  \n*Length: Hip-length  \n*Collar Style: Button-down  \n*Sleeve Style: Long  \n*Patterns: Geometric  \n*Patterns placement: All over  \n*Fluidity of fabric: High  \n*Fabric weight: Light  \n*Pocket Presence: No  \n*Pocket placement: Not applicable  \n*Pocket size: Not applicable  \n*Breathability: High  \n*Occasion Suitability: Business casual  \n*Lapel: Not applicable  '

In [ ]:
cloth_form.replace("  "," ")

'*Type: Shirt \n*Fit: Regular fit \n*Sleeve style: Long sleeve \n*Neckline: Button-down \n*Material: Silk \n*Formality: Business Casual \n*Season: All seasons \n*Colors: 50% pink, 50% white \n*Texture: Smooth \n*Transparency: Opaque \n*Details and Embellishments: Buttons \n*Shape: Fitted \n*Length: Hip-length \n*Collar Style: Button-down \n*Sleeve Style: Long \n*Patterns: Geometric \n*Patterns placement: All over \n*Fluidity of fabric: High \n*Fabric weight: Light \n*Pocket Presence: No \n*Pocket placement: Not applicable \n*Pocket size: Not applicable \n*Breathability: High \n*Occasion Suitability: Business casual \n*Lapel: Not applicable '

## Add to db

In [76]:
conn, cursor = SQLf.sql_db_functions.connect_sql()

In [41]:
# Drop the Table1 if it exists
cursor.execute("DROP TABLE IF EXISTS product_characteristics")

# Create the Table1 with 2 columns ID and Name
cursor.execute("""
    CREATE TABLE product_characteristics (
    id SERIAL PRIMARY KEY,
    Brand_id INTEGER REFERENCES Products(Brand_Prod_id),
    Detail TEXT
    )
""")

# Commit the changes
conn.commit()

In [44]:
query = """
    INSERT INTO product_characteristics (Brand_id, Detail) VALUES (%s, %s)
    RETURNING Brand_id;
    """
cursor.execute(query, (22,'aaaaaaaa'))

# Fetch the rows
rows = cursor.fetchall()
rows

[(22,)]

In [77]:
query = f"SELECT * FROM product_characteristics ;" #LIMIT 5
cursor.execute(query)

# Fetch the rows
rows = cursor.fetchall()
rows

[(2,
  1,
  '\n        image_link \n\n        Description: 0    Il monogramma GG è riproposto in versioni e de...\nName: Descript, dtype: object \n\n        \n        \n\n    The images are of the same clothing peace, use them to fill the following format to list attributes of the clothing peace in image, separate them with comma:  \n\n    *Type:  \n    *Fit: (Slim fit, regular fit, oversized, tight, loose, etc)\n    *Sleeve style: (Short sleeve, long sleeve, sleeveless, cap sleeve, etc)     \n    *Neckline: (Crew neck, V-neck, scoop neck, etc)   \n    *Material: (Cotton, linen, silk, polyester, etc)\n    *Formality: (Casual, Business Casual, Formal, etc)\n    *Seasson:   \n    *Colors:  (percentage of each color in the clothing peace) \n    *Texture: (Smooth, Rough, Ribbed, Velvety, etc)\n    *Transparency: (Opaque, Semi-transparent, Transparent, etc)\n    *Details and Embellishments: (e.g, buttons, zippers, embroidery)\n    *Shape: (e.g., boxy, fitted, flared) \n    *Length: (e.g., c

In [108]:
cursor.execute("DELETE FROM product_characteristics")

In [110]:
conn.commit()

In [75]:
SQLf.sql_db_functions.close_connection_db(conn=conn,cursor=cursor)

In [114]:
len(df) % 10

7